In [2]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [8]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Josh Allen,BUF,QB,5,146.5
1,Nick Chubb,CLE,RB,5,111.0
2,Lamar Jackson,BAL,QB,5,130.1
3,Jalen Hurts,PHI,QB,5,129.0
4,Saquon Barkley,NYG,RB,5,105.6
...,...,...,...,...,...
304,Jonnu Smith,NWE,TE,4,12.8
312,David Bell,CLE,WR,5,10.3
320,Quintin Morris,BUF,TE,4,10.7
321,Albert Okwuegbunam,DEN,TE,5,12.0


### 2. Get Projections

##### 2a. My Projections

In [9]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [17]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}

games_played = {'ARI' : 5,
 'ATL' : 5,
 'BAL' : 5,
 'BUF' : 5,
 'CAR' : 5,
 'CHI' : 5,
 'CIN' : 5,
 'CLE' : 5,
 'DAL' : 5,
 'DEN' : 5,
 'DET' : 5,
 'GNB' : 5,
 'HOU' : 5,
 'IND' : 5,
 'JAX' : 5,
 'KAN' : 4,
 'LAC' : 5,
 'LAR' : 5,
 'LVR' : 4,
 'MIA' : 5,
 'MIN' : 5,
 'NWE' : 5,
 'NOR' : 5,
 'NYG' : 5,
 'NYJ' : 5,
 'PHI' : 5,
 'PIT' : 5,
 'SEA' : 5,
 'SFO' : 5,
 'TAM' : 5,
 'TEN' : 5,
 'WAS' : 5,
 '2TM' : 5}


In [18]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

In [19]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = pd.to_numeric(newerPts['Tm'].replace(games_played))
    newerPts['PPR_adj'] = newerPts['PPR'] * newerPts['GP']
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
# newerPts.to_excel('test.xlsx')

In [20]:
newerPts['Points'].isnull().sum()

0

In [21]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.48349501849845966
0.4009420015639403
0.5035089543816182


0      732.5
1      555.0
2      650.5
3      645.0
4      528.0
       ...  
288     64.0
289     51.5
290     53.5
291     60.0
292     59.0
Name: PPR_adj, Length: 293, dtype: float64